# Using pretrained MobileNetV2

Here we want to use a pre-trained mobile net and train it on the hand images from before. If we use the model already implemented in keras, we can use the weights from ImageNet.

In [31]:
import tensorflow as tf
from tensorflow import keras

from keras import backend as K
from keras.layers import Dense, Activation
from keras.layers import Dropout

from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications import MobileNetV2
from keras.applications import ResNet50

from keras.applications.mobilenet_v2 import MobileNetV2

#from PIL import Image # used for loading images
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam

In [37]:
base_model=VGG16(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3

preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

#specify the inputs
#specify the outputs
model=Model(inputs=base_model.input,outputs=preds)
#now a model has been created based on our architecture

In [3]:
for i,layer in enumerate(model.layers):
   # print(i,layer.name)
    1

Just for comparison, the original mobileNetV2 below

In [22]:
compare_model=MobileNetV2(weights='imagenet',include_top=True) #imports the mobilenet model and discards the last 1000 neuron layer.
for i,layer in enumerate(compare_model.layers):
    #print(i,layer.name)
    1

## What happens when removing the last dense layers (on top)?

the weights in a convolutional layer are fixed-size. They are the size of the kernel x filters. Example: a 3x3 kernel of 10 filters. A convolutional layer doesn't care about the size of the input image. It just does the convolutions and present a resulting image based on the size of the input image. (Search for some illustrated tutorials about convolutions if this is unclear)

now the weights in a dense layer are totally dependent on the input size. It's one weight per element of the input. So this demands that your input be always the same size, or else you won't have proper learned weights.

Because of this, removing the final dense layers allows you to define the input size (see in documentation). (And the output size will increase/decrease accordingly)

Thanks to great stackoverflower [Daniel Möller](https://stackoverflow.com/users/2097240/daniel-m%c3%b6ller)

## Fixing the layers to not be trainable

The model is set up like a numpy array where you can reach each layer by itself. You can set now all layers of the base model frozen.
important is that after setting layer trainable =False, the model has to `be compile()` to take effect.


See Keras [webside](https://keras.io/getting-started/faq/#how-can-i-freeze-keras-layers)

In [23]:
n=100 #149 are all layers before dense output
for layer in model.layers[:n]:
    layer.trainable=False
for layer in model.layers[n:]:
    layer.trainable=True

In [38]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])



For optimal throughput I used a double-buffering approach where the next request is already being prepared (by the CPU) while the current one is still being processed (by the GPU). This way the CPU and GPU are never waiting for one another.

(Fun fact: for V2 it was actually worth doing triple buffering but for V1 that made no difference in speed. This shows that V2 is much more efficient.)

---

Now we use the generator function again to load the data for training

In [25]:
datagen =ImageDataGenerator()

train_batch_generator =datagen.flow_from_directory(directory="C:/Users/werth/Pictures/small_structured/input/Train/",
                                     classes=['Faust', 'Offen'],
                                     target_size=[170,255],
                                     class_mode='categorical',
                                     batch_size=8,
                                     color_mode='rgb',
                                     shuffle=True,
                                     seed=42)

valid_batch_generator =datagen.flow_from_directory(directory="C:/Users/werth/Pictures/small_structured/input/Test/",
                                     classes=['Faust', 'Offen'],
                                     target_size=[170,255],
                                     class_mode='categorical',
                                     batch_size=5,
                                     color_mode='rgb',
                                     shuffle=True,
                                     seed=42)

#img_batch = next(batches)



Found 264 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [8]:
writer = tf.summary.FileWriter( "./graphs", tf.get_default_graph())

In [39]:
%%time
step_size_train=train_batch_generator.n//train_batch_generator.batch_size
step_size_valid=valid_batch_generator.n//valid_batch_generator.batch_size

history=model.fit_generator(generator=train_batch_generator,
                    steps_per_epoch=step_size_train,
                    validation_steps=2,
                    validation_data=valid_batch_generator,
                    epochs=5)

Epoch 1/5
25/33 [=====================>........] - ETA: 39s - loss: 7.5602 - categorical_accuracy: 0.5000

KeyboardInterrupt: 

In [10]:
%tensorboard --logdir=.\logs

UsageError: Line magic function `%tensorboard` not found.


In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')